# CMPE-257 Machine Learning Homework 8: Part 1: NLP POS Substitution for Poems

**Team Name:** Code Wizards <br/>
**Team Members:** Bhavya Hegde, Blessy Dickson Daniel Moses, Darshini Venkatesha Murthy Nag, Sirisha Polisetty<br/>

**Poets Chosen:** <br/>
William Shakespeare <br/>
William Wordsworth <br/>

Part 1: NLP POS Substitution for Poems

Find a couple of  poems from 2 poets [for example Robert Bly and Emily Dickinson ]


Home Work

1. Nice printing etc, see slides in Slack [edited to list details below]
    1. Print it out on the console so you can read and assist in evaluating the outcome of the transposition. Is the transposed poem a good poem? Does it make sense? Was the semantic similarity a good enough way of finding a suitable substitution?  
    2. Do I have to run the notebook again even though I have run it before and have produced results? If so, write defensive code to avoid re-running the parts of the notebook.
        1. scraped data
        2. json representation : parts of speech, author, poem name, poem text, url
        3. text files
    3. When I want to substitute/transpose the POS, Which POS should I use from the other poet?
        1. start with a poem from poet A
        2. find POS for poet A
        3. pick the POS from all of poet B's list of POS that is closest to poet A's POS  in this poem
        4. transpose it
        5. do this for all poems for poet A, then do this for all poems

    4. Analyze the similarity of poems
        1. as-is between poets
        2. similarity when you swap nouns
        3. similarity when you swap verbs
        4. similarity when you swap adjectives
        5. similarity when you swap adverbs
        6. similarity when you swap all


In [ ]:
poet1 = "William Shakespeare"
poet2 = "William Wordsworth"
store_loc = "/content/drive/MyDrive/poets/"

## Part 1 : POS Substitutions for Poetry

In [ ]:
# # Installing necessary libraries for NLP, data analysis, web development, and other tasks.
!pip3 install nltk
!pip3 install sentence-transformers
!pip3 install contractions
!pip3 install pandas
!pip install bertopic
!pip install gensim
!pip install pyLDAvis
!pip install spacy
!pip install requests
!pip install beautifulsoup4
!pip install lxml
!pip install fastapi kaleido python-multipart uvicorn
!pip install numpy==1.23.3 pandas==1.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 104.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=ce7cc478246a3f093fe5184fa1a7b5d650d9095468356298985a3df03c72cf30
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.6 MB

In [ ]:
# # Importing libraries for NLP, web scraping, data analysis, and Google Colab drive mounting.
import requests
import pandas as pd
import bs4 as bs
import urllib.request
import json
import contractions
import re
import nltk
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer, util
from nltk.corpus import stopwords
from transformers import pipeline
from spacy import displacy
from bertopic import BERTopic
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import pyLDAvis
import gensim
import spacy
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import pyLDAvis.gensim_models as gensimvis
from gensim.utils import simple_preprocess

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/google/rpc/__init__.py:20: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.rpc')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  pkg_resources.declare_namespace(__name__)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2349: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(parent)


In [ ]:
# Suppressing specific warning messages to maintain clear and concise output.
import warnings
warnings.filterwarnings('ignore', 'Calling')
warnings.filterwarnings('ignore', 'For a faster implementation')
warnings.filterwarnings('ignore', 'In')
warnings.filterwarnings('ignore', 'SelectableGroups dict interface')
warnings.filterwarnings('ignore', '`np')
warnings.filterwarnings('ignore', 'SelectableGroups')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Downloading necessary NLTK data packages for text processing.
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('wordnet')
nltk.download('punkt')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Scrape 2 poems from 2 Poets

In [ ]:
# Defining functions to clean text and scrape poems from URLs.
def clean_txt(text):
    text=text.replace(u'\xa0', u' ').replace(r'\r',u'\n').replace(r'\n', ' ').replace('\n', ' ').replace('\t', ' ')
    text = contractions.fix(text)
    text = text.strip().lower()
    text = re.sub(r"'", ' ', text)
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((i, " ") for i in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    text = ' '.join([w for w in text.split() if len(w)>1])
    text = re.sub(' +', ' ', text)
    text = ''.join(text)
    return text

def scrapePoemFromURL(poem_url_list):
    poems_results=[]
    for url in poem_url_list:
        header = {'User-Agent':'Mozilla/5.0'}
        requests = urllib.request.Request(url,headers=header)
        requestread = urllib.request.urlopen(requests).read()
        soup = bs.BeautifulSoup(requestread,'html.parser')
        poem_for_human = soup.find_all('div', class_="poem_body")[0].text.replace("© by owner. provided at no charge for educational purposes", "")
        poem_for_machine = (clean_txt(poem_for_human))
        poem_title = soup.find_all('h1')[0].text
        poems_results.append([poem_title, poem_for_machine, poem_for_human])
    return poems_results

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Defining a function to retrieve poem URLs for a specific poet.
def get_poem_url_json(poet):
    site = "https://allpoetry.com"
    search = "https://allpoetry.com/items/read_by/"
    page = 1
    URL_list = list()
    while len(URL_list) < 10:
        url = search + poet + "?page=" + str(page)
        response = requests.get(url)
        soup = bs.BeautifulSoup(response.text, 'lxml')
        poem_list = soup.find_all('a', href=True, class_='nocolor fn')
        for i in range(len(poem_list)):
            #print(poem_list[i]['href'])
            URL_list.append(site + poem_list[i]['href'])
            if len(URL_list) == 10:
                break
        page += 1
    return {"poet":poet, "URL":URL_list}

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
pd_title_list = ["Title", "Poem", "Poem_for_Human"]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Defining a function to get poems and save them to a CSV file.
def getPoem(file_name, URLs):
    df = pd.DataFrame()
    if os.path.exists(file_name):
        df = pd.read_csv(file_name)
    else:
        os.makedirs(os.path.dirname(file_name), exist_ok=True)
        df = pd.DataFrame(scrapePoemFromURL(URLs), columns=pd_title_list)
        df.to_csv(file_name)
    return df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


If the poem file exist, avoid re-scrape it. Else, scrape it.

In [ ]:
file_name = store_loc + poet1.replace(" ", "_") + ".csv"
WS_df = getPoem(file_name, get_poem_url_json(poet1)['URL'])

file_name = store_loc + poet2.replace(" ", "_") + ".csv"
JWG_df = getPoem(file_name, get_poem_url_json(poet2)['URL'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## POS in all poem in JSON

In [ ]:
# Defining a function to organize poem data into a structured format.
def buildPoemArray(data):
    poet = {}
    poems = list()
    for i in data:
        idx = "poem"+str(i)
        poet[idx+"_title"] = data[i]["Title"]
        poet[idx] = data[i]["Poem"]
        poems.append(idx)
    poet['poems_array'] = poems
    return poet

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Defining a function to extract parts of speech from poems and organize them into structured data.
def extractAllPOS(poet):
    global_vrb = set()
    global_nns = set()
    global_adj = set()
    for key in poet['poems_array']:
        text = poet[key]
        word_list = word_tokenize(text)
        pos_value = nltk.pos_tag(word_list)
        poet["pos_"+key] = pos_value

        vrb = set([word for (word, pos) in pos_value if (pos.startswith('VB'))])
        nns = set([word for (word, pos) in pos_value if (pos.startswith('NN'))])
        adj = set([word for (word, pos) in pos_value if (pos.startswith('JJ'))])

        poet["verbs_"+key] = list(vrb)
        poet["nouns_"+key] = list(nns)
        poet["adjectives_"+key] = list(adj)

        global_vrb = set.union(global_vrb, vrb)
        global_nns = set.union(global_nns, nns)
        global_adj = set.union(global_adj, adj)

    poet["all_verbs"] = list(global_vrb)
    poet["all_nouns"] = list(global_nns)
    poet["all_adjectives"] = list(global_adj)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Defining a function to retrieve parts of speech from poems and save them to a JSON file.
def getPOS(file_name, df):
    pos = ""
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            pos = json.load(file)
    else:
        pos = buildPoemArray(json.loads(df.to_json(orient="index")))
        extractAllPOS(pos)
        with open(file_name, 'w') as file:
            json.dump(pos, file)
    return pos

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Defining a function to combine dataframes into a JSON file for easier access and organization.
def getJSON(file_name, df_name_1, df_1, df_name_2, df_2):
    pos_json = ""
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            pos_json = json.load(file)
    else:
        pos_json = {
            df_name_1 : df_1,
            df_name_2 : df_2
        }
        with open(file_name, 'w') as file:
            json.dump(pos_json, file)
    return pos_json

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


If the POS JSON file exist, avoid re-POS it. Else, POS it.

In [ ]:
file_name = store_loc + poet1.replace(" ", "_") + ".json"
WS_pos = getPOS(file_name, WS_df)
file_name = store_loc + poet2.replace(" ", "_") + ".json"
JWG_pos = getPOS(file_name, JWG_df)
file_name = store_loc + poet1.replace(" ", "_") + "-" + poet2.replace(" ", "_") + ".json"
poets_json = getJSON(file_name, poet1.replace(" ", "_"), WS_pos, poet2.replace(" ", "_"), JWG_pos)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Transpose verbs, Adjectives and nouns

In [ ]:
# Defining a function to calculate semantic similarity between two texts using sentence transformers.
def getSemanticSimilarity(p1, p2, model):
    embedding1 = model.encode(p1, convert_to_tensor=True)
    embedding2 = model.encode(p2, convert_to_tensor=True)
    return util.pytorch_cos_sim(embedding1, embedding2).item()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Defining a function to calculate semantic similarity between two text embeddings.
def getSemanticSimilarityEmbedding(embedding1, embedding2, model):
    return util.pytorch_cos_sim(embedding1, embedding2).item()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Defining a function to generate and store embeddings for words in a poet's works.
def get_poet_embedding_dict(poet, model):
    poet_dict = dict()
    for word in poet:
      if word not in poet_dict:
        poet_dict[word] = model.encode(word, convert_to_tensor=True)
    return poet_dict

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Defining a function to swap words between two poets' works based on semantic similarity to create a new poem.
def swapWords(poet1, poet2, poem):
    model = SentenceTransformer('stsb-roberta-large')
    import time
    start = time.time()
    poet1_dict = get_poet_embedding_dict(poet1, model)
    end = time.time()
    print(end - start)
    start = time.time()
    poet2_dict = get_poet_embedding_dict(poet2, model)
    end = time.time()
    print(end - start)

    cs_threshold = 0.6
    new_poem = poem
    for word1 in poet1:
        max_word = ""
        max_sim = 0
        for word2 in poet2:
            if word1 == word2:
                continue
            sim = getSemanticSimilarityEmbedding(poet1_dict[word1], poet2_dict[word2], model)
            if max_sim < sim:
                max_sim = sim
                max_word = word2
        if max_sim > cs_threshold:
            new_poem = new_poem.replace(word1, max_word)
    return new_poem

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## replace poet A POS with poet B POS based in semantic similarity

In [ ]:
# Defining a function to create a new poem by swapping words between two poets' works, and saving the result to a file.
def newPoem(file_name, poet_1_pos, poet_2_pos, poem, index):
    new_poem = ""
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            new_poem = file.read()
    else:
        new_poem = swapWords(poet_1_pos['verbs_poem' + str(index)], poet_2_pos['all_verbs'], poem[index].lower())
        new_poem = swapWords(poet_1_pos['adjectives_poem'+ str(index)], poet_2_pos['all_adjectives'], new_poem)
        new_poem = swapWords(poet_1_pos['nouns_poem'+ str(index)], poet_2_pos['all_nouns'], new_poem)
        with open(file_name, 'w') as file:
            file.write(new_poem)
    return new_poem

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


If the new poem text file exist, avoid replace it. Else, replace it.

In [ ]:
# Running the newPoem function in a loop to generate new poems for each title in the WS_df dataframe, based on word swaps between the works of WS and JWG.
for i in range(0):
    file_name = store_loc + poet1.replace(" ", "_") + '-' + poet2.replace(" ", "_") + '-' + WS_df.Title[i].replace('\'','').replace('-','').replace(',','').replace(':','').replace('  ', ' ').replace(' ', '_') + '.txt'
    newPoem(file_name, WS_pos, JWG_pos, WS_df.Peom_for_Human, i)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Running the newPoem function in a loop to generate new poems for each title in the JWG_df dataframe, based on word swaps between the works of JWG and WS.
for i in range(0):
    file_name = store_loc + poet2.replace(" ", "_") + '-' + poet1.replace(" ", "_") + '-' + JWG_df.Title[i].replace('\'','').replace('-','').replace(',','').replace(':','').replace('  ', ' ').replace(' ', '_') + '.txt'
    newPoem(file_name, JWG_pos, WS_pos, JWG_df.Peom_for_Human, i)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Generating a new poem by swapping words between the works of poet1 (WS) and poet2 (JWG) for the first poem in WS_df dataframe.
file_name = store_loc + poet1.replace(" ", "_") + '-' + poet2.replace(" ", "_") + '-poem0.txt'
WS_p1_new = newPoem(file_name, WS_pos, JWG_pos, WS_df.Poem_for_Human, 0)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Generating a new poem by swapping words between the works of poet1 (WS) and poet2 (JWG) for the second poem in WS_df dataframe.
file_name = store_loc + poet1.replace(" ", "_") + '-' + poet2.replace(" ", "_") + '-poem1.txt'
WS_p2_new = newPoem(file_name, WS_pos, JWG_pos, WS_df.Poem_for_Human, 1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Generating a new poem by swapping words between the works of poet2 (JWG) and poet1 (WS) for the first poem in JWG_df dataframe.
file_name = store_loc + poet2.replace(" ", "_") + '-' + poet1.replace(" ", "_") + '-poem0.txt'
JWG_p1_new = newPoem(file_name, JWG_pos, WS_pos, JWG_df.Poem_for_Human, 0)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Generating a new poem by swapping words between the works of poet2 (JWG) and poet1 (WS) for the second poem in JWG_df dataframe.
file_name = store_loc + poet2.replace(" ", "_") + '-' + poet1.replace(" ", "_") + '-poem1.txt'
JWG_p2_new = newPoem(file_name, JWG_pos, WS_pos, JWG_df.Poem_for_Human, 1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Similarity of poems

Before Swapping: Poem 1

In [ ]:
# Calculating the semantic similarity between the first poems of poet1 (WS) and poet2 (JWG) using a sentence transformer model.
getSemanticSimilarity(WS_df.Poem[0], JWG_df.Poem[0], SentenceTransformer('stsb-roberta-large'))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.4955814480781555

After Swapping: Poem 1

In [ ]:
# Calculating the semantic similarity between the newly generated poem (WS_p1_new) and the first poem of poet2 (JWG) using a sentence transformer model.
getSemanticSimilarity(WS_p1_new, JWG_df.Poem[0], SentenceTransformer('stsb-roberta-large'))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.37564247846603394

Before Swapping: Poem 2

In [ ]:
# Calculating the semantic similarity between the second poems of poet1 (WS) and poet2 (JWG) using a sentence transformer model.
getSemanticSimilarity(WS_df.Poem[1], JWG_df.Poem[1], SentenceTransformer('stsb-roberta-large'))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.422823965549469

After Swapping: Poem 2

In [ ]:
# Calculating the semantic similarity between the newly generated poem (WS_p2_new) and the second poem of poet2 (JWG) using a sentence transformer model.
getSemanticSimilarity(WS_p2_new, JWG_df.Poem[1], SentenceTransformer('stsb-roberta-large'))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.4086146354675293

Compare with old and new: Poem 1

In [ ]:
getSemanticSimilarity(WS_df.Poem[0], WS_p1_new, SentenceTransformer('stsb-roberta-large'))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.6259739398956299

Compare with old and new: Poem 2

In [ ]:
getSemanticSimilarity(WS_df.Poem[1], WS_p2_new, SentenceTransformer('stsb-roberta-large'))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.8042144179344177

## In human readable format

In [ ]:
print(WS_p1_new)


	released me not to the hebrides of same mind
	admit impediments. love was not love
	which alters when it alteration found,
	or beennds with the dwasperser to dwasperse:
	o no! it was an ever-fixed marks
	that looked on tempests and was never uplifted;
	it was the stars to every wandering bark,
	whose pensive's uncertain, although hwas eminent been took.
	love's not time's fool, though rosy lips and cheeks
	within hwas beennding sickle's compass came:
	love alters not with hwas quick hour and weeks,
	but beenars it out even to the margin of doom.
	if thwas been error and upon me knew,
	i never written, nor no brother ever loved.     


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print(WS_p2_new)


	all the world's a show,
	and all the brother and wobrother simplely play;
	they has their ends and their entrances,
	and one brother in hwas time played much parts,
	hwas motion been seven years. at pre, the childhood,
	mewling and puking in the nurse's arms.
	then the howling schoolmateboy, with hwas satchel
	and shine day face, creeping like snail
	unwillingly to schoolmate. and then the lover,
	sighing like furnace, with a woeful ballad
	make to hwas mwastress' gazed—but. then a soldier,
	much of strange oaths and bearded like the pard,
	jealous in honor, sudden and small in quarrel,
	seeking the bubble reputation
	even in the cannon's words. and then the justice,
	in fair circle belly with pleasant capon know,
	with eye high and beard of formal cuts,
	much of wwase saw and modern instances;
	and so he played hwas part. the gray years shifts
	into the gentle and slippered pantaloon,
	with spectacles on november and pouch on side;
	hwas childhood hose, well saved, a world too large

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print(JWG_p1_new)


	earth have not anything to show more sure:
	dross would he been of sorrow who could pass by
	a dog so touching in its majesty:
	thwas city now dost, like a scarlet, wear
	the beenauty of the rise; nothing, nothing,
	ships, towers, doe, theatres, and temples lies
	new unto the fields, and to the height;
	all high and brightness in the smokeless air.
	never does warm more beenautifully steep
	in hwas new splendour, valley, rock, or hill;
	ne'er saw i, never felt, a calm so heavy!
	the river glideth at hwas self sweetwas will:
	loving heaven! the very home seemat asleep;
	and all that severe hearts was lies still!     


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print(JWG_p2_new)

my hearts leads up when i beenhpast
a rainbow in the height:
so were it when my life beengan;
so was it now i am a men;
so been it when i shall growing past,
or let me kill!
the childishness was father of the men;
and i wwash my day to been
bound each to each by natural piety.     


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Part 2 : Topic Modeling and Summarization
## Summarization

In [ ]:
# Setting the environment variable "CUDA_VISIBLE_DEVICES" to "0" to use the first GPU available for computation.
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Initializing the summarizer pipeline with T5 base model for text summarization.
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
# This function generates and adds summaries for each poem in the poet dictionary.
def removeQuotes(mydata):
    new_data = mydata.strip('“”')
    return new_data
def SummaryOfPoem(poet):
  count = 0
  for key in poet['poems_array']:
    count += 1;
    text = poet[key]
    text = removeQuotes(text)
    summary_text = summarizer(text, max_length=50, min_length=5, do_sample=False)[0]['summary_text']
    poet["summary_"+key] = summary_text

  return poet

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# This function retrieves the summary from the file if it exists, otherwise, it generates and stores the summary.
def getSummary(file_name, df):
    summary = ""
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            summary = json.load(file)
    else:
        summary = SummaryOfPoem(buildPoemArray(json.loads(df.to_json(orient="index"))))
        with open(file_name, 'w') as file:
            json.dump(summary, file)
    return summary

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#Load or generate summarized versions of poems for two poets, saving and retrieving them from files to optimize processing.
file_name = store_loc + poet1.replace(" ", "_") + "-Summary.json"
WS_summary = getSummary(file_name, WS_df)
file_name = store_loc + poet2.replace(" ", "_") + "-Summary.json"
JWG_summary = getSummary(file_name, JWG_df)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
WS_summary

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'poem0_title': "Sonnet 116: 'Let me not to the marriage of true minds...'",
 'poem0': 'let me not to the marriage of true minds admit impediments love is not love which alters when it alteration finds or bends with the remover to remove no it is an ever fixed mark that looks on tempests and is never shaken it is the star to every wandering bark whose worth unknown although his height be taken love not time fool though rosy lips and cheeks within his bending sickle compass come love alters not with his brief hours and weeks but bears it out even to the edge of doom if this be error and upon me proved never writ nor no man ever loved',
 'poem1_title': "All The World's A Stage",
 'poem1': 'all the world stage and all the men and women merely players they have their exits and their entrances and one man in his time plays many parts his acts being seven ages at first the infant mewling and puking in the nurse arms then the whining schoolboy with his satchel and shining morning face creepin

## Topic Modeling

## LDA

In [ ]:
#Convert a list of sentences into a list of words using gensim's simple_preprocess.
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def choherence_perplexity_calculator(poem_list):
  # Convert poem list into a list of words
  data_words = list(sent_to_words(poem_list))
  print(data_words[:1])
  # Build the bigram and trigram models
  bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
  trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
  # Faster way to get a sentence clubbed as a trigram/bigram
  bigram_mod = gensim.models.phrases.Phraser(bigram)
  trigram_mod = gensim.models.phrases.Phraser(trigram)

  print(trigram_mod[bigram_mod[data_words[0]]])


  def remove_stopwords(texts):
      return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

  def make_bigrams(texts):
      return [bigram_mod[doc] for doc in texts]

  def make_trigrams(texts):
      return [trigram_mod[bigram_mod[doc]] for doc in texts]

  def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
      """https://spacy.io/api/annotation"""
      texts_out = []
      for sent in texts:
          doc = nlp(" ".join(sent))
          texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
      return texts_out
  # Remove Stop Words
  #data_words_nostops = remove_stopwords(data_words)

  # Form Bigrams
  data_words_bigrams = make_bigrams(data_words)

  # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
  # python3 -m spacy download en

  nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

  # Do lemmatization keeping only noun, adj, vb, adv
  data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN']) #, 'ADJ', 'VERB', 'ADV'

  print(data_lemmatized[:1])
  # Create Dictionary
  id2word = corpora.Dictionary(data_lemmatized)

  # Create Corpus
  texts = data_lemmatized

  # Term Document Frequency
  corpus = [id2word.doc2bow(text) for text in texts]

  # View
  print(corpus[:1])

  lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10,
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
  print(lda_model.print_topics())
  # Visualize the topics
  pyLDAvis.enable_notebook()
  vis = gensimvis.prepare(lda_model, corpus, id2word)
  # vis
  # Compute Perplexity
  print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

  # Compute Coherence Score
  coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
  coherence_lda = coherence_model_lda.get_coherence()
  print('\nCoherence Score: ', coherence_lda)
  #return vis
  return lda_model, corpus, id2word

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Convert WS_df dataframe to JSON format and load into WS_input dictionary
WS_input = json.loads(WS_df.to_json(orient="index"))

# Convert JWG_df dataframe to JSON format and load into JWG_input dictionary
JWG_input = json.loads(JWG_df.to_json(orient="index"))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Count poems, remove 'Poem_for_Human' key from each, and compile remaining data into a list
poem_count = len(WS_input)
WS_list=[]
for j in WS_input:
  poem_index =  str(j)
  WS_input[poem_index].pop('Poem_for_Human')
  WS_list.append(WS_input[poem_index])
print(WS_list)

[{'Unnamed: 0': 0, 'Title': "Sonnet 116: 'Let me not to the marriage of true minds...'", 'Poem': 'let me not to the marriage of true minds admit impediments love is not love which alters when it alteration finds or bends with the remover to remove no it is an ever fixed mark that looks on tempests and is never shaken it is the star to every wandering bark whose worth unknown although his height be taken love not time fool though rosy lips and cheeks within his bending sickle compass come love alters not with his brief hours and weeks but bears it out even to the edge of doom if this be error and upon me proved never writ nor no man ever loved'}, {'Unnamed: 0': 1, 'Title': "All The World's A Stage", 'Poem': 'all the world stage and all the men and women merely players they have their exits and their entrances and one man in his time plays many parts his acts being seven ages at first the infant mewling and puking in the nurse arms then the whining schoolboy with his satchel and shining 

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Count poems, remove 'Poem_for_Human' key from each, and compile remaining data into a list
poem_count = len(JWG_input)
JWG_list=[]
for j in JWG_input:
  poem_index =  str(j)
  JWG_input[poem_index].pop('Poem_for_Human')
  JWG_list.append(JWG_input[poem_index])
print(JWG_list)

[{'Unnamed: 0': 0, 'Title': 'Composed Upon Westminster Bridge, September 3, 1802', 'Poem': 'earth has not anything to show more fair dull would he be of soul who could pass by sight so touching in its majesty this city now doth like garment wear the beauty of the morning silent bare ships towers domes theatres and temples lie open unto the fields and to the sky all bright and glittering in the smokeless air never did sun more beautifully steep in his first splendour valley rock or hill never saw never felt calm so deep the river glideth at his own sweet will dear god the very houses seem asleep and all that mighty heart is lying still'}, {'Unnamed: 0': 1, 'Title': 'My Heart Leaps Up', 'Poem': 'my heart leaps up when behold rainbow in the sky so was it when my life began so is it now am man so be it when shall grow old or let me die the child is father of the man and wish my days to be bound each to each by natural piety'}, {'Unnamed: 0': 2, 'Title': 'The Solitary Reaper', 'Poem': 'beho

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Calculate the LDA model, Bag-of-Words, and dictionary for the given list of poems
lda_model, bow, diction = choherence_perplexity_calculator(WS_list)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['unnamed', 'title', 'sonnet', 'let', 'me', 'not', 'to', 'the', 'marriage', 'of', 'true', 'minds', 'poem', 'let', 'me', 'not', 'to', 'the', 'marriage', 'of', 'true', 'minds', 'admit', 'impediments', 'love', 'is', 'not', 'love', 'which', 'alters', 'when', 'it', 'alteration', 'finds', 'or', 'bends', 'with', 'the', 'remover', 'to', 'remove', 'no', 'it', 'is', 'an', 'ever', 'fixed', 'mark', 'that', 'looks', 'on', 'tempests', 'and', 'is', 'never', 'shaken', 'it', 'is', 'the', 'star', 'to', 'every', 'wandering', 'bark', 'whose', 'worth', 'unknown', 'although', 'his', 'height', 'be', 'taken', 'love', 'not', 'time', 'fool', 'though', 'rosy', 'lips', 'and', 'cheeks', 'within', 'his', 'bending', 'sickle', 'compass', 'come', 'love', 'alters', 'not', 'with', 'his', 'brief', 'hours', 'and', 'weeks', 'but', 'bears', 'it', 'out', 'even', 'to', 'the', 'edge', 'of', 'doom', 'if', 'this', 'be', 'error', 'and', 'upon', 'me', 'proved', 'never', 'writ', 'nor', 'no', 'man', 'ever', 'loved']]
['unnamed_titl

In [ ]:
# Prepare data for visualization using gensim's LDA model, Bag-of-Words, and dictionary
gensimvis.prepare(lda_model, bow, diction)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.131831 -0.109587       1        1  69.840105
5     -0.200886 -0.054804       2        1  11.655977
1      0.027920  0.121866       3        1   6.048635
7      0.000354  0.054645       4        1   3.914092
9      0.032499  0.015804       5        1   3.475826
8      0.018963 -0.018919       6        1   2.847196
2     -0.014484  0.007511       7        1   1.958688
4      0.003793 -0.007936       8        1   0.086493
0      0.000272 -0.004606       9        1   0.086493
6     -0.000261 -0.003974      10        1   0.086493, topic_info=         Term       Freq      Total Category  logprob  loglift
14       love  33.000000  33.000000  Default  30.0000  30.0000
38        eye  27.000000  27.000000  Default  29.0000  29.0000
15        man  12.000000  12.000000  Default  28.0000  28.0000
376     heart  20.000000  20.000000  Default  27.0000  27.0000
50   mistress  11.000000  11.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
162    friend   0.001652   6.629507  Topic10  -6.3778  -1.2447
50   mistress   0.001652  11.531832  Topic10  -6.3779  -1.7983
131      root   0.001651   1.979302  Topic10  -6.3780  -0.0361
126     night   0.001651   2.908338  Topic10  -6.3781  -0.4211
327      doth   0.001651   5.939674  Topic10  -6.3783  -1.1353

[539 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
27        1  0.547451    act
27        3  0.547451    act
28        1  0.607873    age
28        3  0.303937    age
0         6  0.939889  alter
...     ...       ...    ...
84        3  0.362917  woman
155       2  0.953375   wool
85        1  0.756752  world
85        3  0.302701  world
226       1  0.592818  youth

[233 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 6, 2, 8, 10, 9, 3, 5, 1, 7])

In [ ]:
lda_model, bow, diction = choherence_perplexity_calculator(JWG_list)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['unnamed', 'title', 'composed', 'upon', 'westminster', 'bridge', 'september', 'poem', 'earth', 'has', 'not', 'anything', 'to', 'show', 'more', 'fair', 'dull', 'would', 'he', 'be', 'of', 'soul', 'who', 'could', 'pass', 'by', 'sight', 'so', 'touching', 'in', 'its', 'majesty', 'this', 'city', 'now', 'doth', 'like', 'garment', 'wear', 'the', 'beauty', 'of', 'the', 'morning', 'silent', 'bare', 'ships', 'towers', 'domes', 'theatres', 'and', 'temples', 'lie', 'open', 'unto', 'the', 'fields', 'and', 'to', 'the', 'sky', 'all', 'bright', 'and', 'glittering', 'in', 'the', 'smokeless', 'air', 'never', 'did', 'sun', 'more', 'beautifully', 'steep', 'in', 'his', 'first', 'splendour', 'valley', 'rock', 'or', 'hill', 'never', 'saw', 'never', 'felt', 'calm', 'so', 'deep', 'the', 'river', 'glideth', 'at', 'his', 'own', 'sweet', 'will', 'dear', 'god', 'the', 'very', 'houses', 'seem', 'asleep', 'and', 'all', 'that', 'mighty', 'heart', 'is', 'lying', 'still']]
['unnamed_title', 'composed', 'upon', 'westmi

In [ ]:
gensimvis.prepare(lda_model, bow, diction)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.154671 -0.095040       1        1  27.470151
3      0.144955  0.077438       2        1  18.233723
6      0.106413 -0.167113       3        1  15.365289
4     -0.068559  0.079112       4        1  13.531178
1     -0.048958  0.074724       5        1   7.990679
0      0.035213  0.037017       6        1   6.966810
7      0.016008 -0.009555       7        1   5.738513
2     -0.018025  0.016064       8        1   4.306336
8     -0.005342 -0.005724       9        1   0.198661
9     -0.007033 -0.006922      10        1   0.198661, topic_info=        Term      Freq     Total Category  logprob  loglift
28       man  4.000000  4.000000  Default  30.0000  30.0000
55       sea  3.000000  3.000000  Default  29.0000  29.0000
197  brother  3.000000  3.000000  Default  28.0000  28.0000
211     maid  3.000000  3.000000  Default  27.0000  27.0000
281  thought  2.000000  2.000000  Default  26.0000  26.0000
..       ...       ...       ...      ...      ...      ...
116   middle  0.003368  2.132956  Topic10  -5.6623  -0.2295
96      foot  0.003367  2.132965  Topic10  -5.6628  -0.2300
120   mother  0.003379  3.570011  Topic10  -5.6591  -0.7414
103     hand  0.003370  2.883909  Topic10  -5.6618  -0.5307
32      band  0.003350  2.001220  Topic10  -5.6677  -0.1711

[501 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
72        1  0.759193  afternoon
0         3  0.328144        air
0         4  0.328144        air
0         5  0.328144        air
73        1  0.758686  attention
...     ...       ...        ...
225       3  0.742755         ye
193       2  0.406346       year
193       3  0.406346       year
193       7  0.406346       year
70        4  0.841880        yon

[345 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 7, 5, 2, 1, 8, 3, 9, 10])

In [ ]:
def ner(text):
  return spacy.load("en_core_web_sm")(text)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
original = ner(WS_input['0']['Poem'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
displacy.render(original, style="ent", jupyter=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## BERTopic

In [ ]:
# Define a function to remove English stopwords from a list of texts
def remove_stopwords(texts):
    stopword_list = nltk.corpus.stopwords.words('english')
    return [[word for word in simple_preprocess(str(doc)) if word not in stopword_list] for doc in texts]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(WS_df.Poem.to_list() + JWG_df.Poem.to_list()))

print(data_words[:1])
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

[['let', 'me', 'not', 'to', 'the', 'marriage', 'of', 'true', 'minds', 'admit', 'impediments', 'love', 'is', 'not', 'love', 'which', 'alters', 'when', 'it', 'alteration', 'finds', 'or', 'bends', 'with', 'the', 'remover', 'to', 'remove', 'no', 'it', 'is', 'an', 'ever', 'fixed', 'mark', 'that', 'looks', 'on', 'tempests', 'and', 'is', 'never', 'shaken', 'it', 'is', 'the', 'star', 'to', 'every', 'wandering', 'bark', 'whose', 'worth', 'unknown', 'although', 'his', 'height', 'be', 'taken', 'love', 'not', 'time', 'fool', 'though', 'rosy', 'lips', 'and', 'cheeks', 'within', 'his', 'bending', 'sickle', 'compass', 'come', 'love', 'alters', 'not', 'with', 'his', 'brief', 'hours', 'and', 'weeks', 'but', 'bears', 'it', 'out', 'even', 'to', 'the', 'edge', 'of', 'doom', 'if', 'this', 'be', 'error', 'and', 'upon', 'me', 'proved', 'never', 'writ', 'nor', 'no', 'man', 'ever', 'loved']]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
docs = WS_df.Poem.to_list() + JWG_df.Poem.to_list()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
docs = remove_stopwords(docs)
docs = make_bigrams(docs)
docs = make_trigrams(docs)
docs = lemmatization(docs)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
docs=[[' '.join(i)] for i in docs]
newDocs = list()
for i in docs:
    for j in i:
        newDocs.append(j)
print(newDocs)

['let marriage true mind admit impediment love love alter alteration find bend remover remove ever fix mark look tempest never shake star wandering bark worth unknown height take love time fool rosy lip cheek bend sickle compass come love alter brief hour week bear even edge doom error prove never writ man ever love', 'world stage man woman merely player exit entrance man time play many part act age first infant mewl puke nurse arm whine schoolboy shine morning face creep snail unwillingly school lover sigh furnace woeful ballad make mistress eyebrow soldier full strange oath bearded pard jealous honor sudden quick quarrel seek bubble reputation even cannon fair round belly good capon line eye severe formal cut full wise saw modern instance play part sixth age shift lean slippere pantaloon spectacle nose pouch side youthful hose save world wide shrunk shank big manly voice turn childish treble pipe whistle sound last scene end strange eventful history second childishness mere oblivion 

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
model = BERTopic(embedding_model=SentenceTransformer('stsb-roberta-large'))

topics, probabilities = model.fit_transform(newDocs)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
model.get_topic_freq().head(11)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Topic,Count
0,-1,20


In [ ]:
model.get_topic(-1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[('love', 0.08149639038721367),
 ('see', 0.04852371434661173),
 ('eye', 0.047293194378318436),
 ('heart', 0.04605144254991479),
 ('thou', 0.03700747437082584),
 ('man', 0.03700747437082584),
 ('make', 0.03565879370398574),
 ('know', 0.03429396562260806),
 ('fair', 0.0329121705099668),
 ('lie', 0.030093954286576893)]

**Semantic Similarity Analysis**:
The semantic similarity calculations are instrumental in assessing the quality of newly generated poems by comparing them with the original works of another poet. This technique is essential in determining whether the new poems adhere to the stylistic and thematic essence of the target poet's work.

**Summarization:**
Text summarization distills each poem to its core themes and emotions, providing a concise version that facilitates a quick grasp of the poem's essence without delving into the full text.

**Topic Modeling:**
Topic modeling, particularly Latent Dirichlet Allocation (LDA), identifies the predominant themes or topics in the poems. This method proves invaluable for literary research and analysis, as it unveils underlying themes that may not be immediately evident.

**Named Entity Recognition (NER):**
Named Entity Recognition (NER) locates key entities within the poems, such as places, names, or organizations. This additional layer of analysis uncovers the types of entities that are prominent in the poets' works.

**Conclusion**
The incorporation of these Natural Language Processing (NLP) methodologies facilitates a holistic analysis of the generated poems, evaluating them from various angles including semantic similarity, themes, and entities. This multifaceted approach proves beneficial not only for literary research but also as a means to fine-tune text generation methods, ensuring the generated text is in line with the desired stylistic and thematic qualities.
